In [1]:

%load_ext autoreload
%autoreload 2

In [7]:
import typer
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from transformers import get_cosine_schedule_with_warmup
from transformers import AutoImageProcessor
import gymnasium as gym

from uav_active_sensing.config import IMG_BATCH_SIZE, DEVICE, MODELS_DIR
from uav_active_sensing.pytorch_datasets import TinyImageNetDataset, tiny_imagenet_collate_fn
from uav_active_sensing.modeling.img_exploration_env import RewardFunction, img_pairs_generator, ImageExplorationEnv, unwrap_env, ImageExplorationEnvConfig
from uav_active_sensing.modeling.act_vit_mae import ActViTMAEForPreTraining
from uav_active_sensing.modeling.configuration_act_vit_mae import ActViTMAETrainingConfig
from uav_active_sensing.modeling.ppo import train_ppo, make_env, PPOConfig, PPOAgent
from uav_active_sensing.utils import load_model_state_dict

import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common import env_checker


In [40]:
# TODO: Custom feature extractor to agent
# https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html#default-network-architecture
# Try light resnet, or something of the kind

In [39]:
# Your image processor and dataset setup
image_processor = AutoImageProcessor.from_pretrained("facebook/vit-mae-base", use_fast=True)
tiny_imagenet_train_dataset = TinyImageNetDataset(split="train", transform=image_processor)
tiny_imagenet_train_loader = DataLoader(tiny_imagenet_train_dataset, batch_size=1, collate_fn=tiny_imagenet_collate_fn)

# Pretrained model and reward function
model = ActViTMAEForPreTraining.from_pretrained("facebook/vit-mae-base")
reward_function = RewardFunction(model)
ppo_config = PPOConfig()

# Create a dummy environment to initialize the model
dummy_batch = next(iter(tiny_imagenet_train_loader))  # Just take one image as a dummy
# dummy_env = vect_env = DummyVecEnv(
#             [make_env(img.unsqueeze(0), reward_function, ppo_config.gamma) for img in dummy_batch]
#         )

env_config = ImageExplorationEnvConfig(img=dummy_batch, reward_function=reward_function)
env = ImageExplorationEnv(env_config)
rl_model = PPO("MlpPolicy", env, verbose=1, policy_kwargs=dict(normalize_images=False))  # Initialize model with dummy environment

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [34]:
for batch in tiny_imagenet_train_loader:
    # Create the environment with the current batch of images
    env.img = batch
    
    # # Reassign the environment to the model with the new batch
    # rl_model.set_env(vect_env)

    # Now the model interacts with the new environment and learns
    rl_model.learn(total_timesteps=100)
    break

torch.Size([1, 3, 224, 224])


ValueError: could not broadcast input array from shape (3,224,224) into shape (224,224,3)